In [13]:
from datetime import datetime

from pyspark.sql import SparkSession, DataFrame
from pyspark.sql.types import StringType
from pyspark.sql.functions import udf, col, split, when, to_date, max, min

In [2]:
# Create a SparkSession
spark = SparkSession.builder \
    .master("local[*]") \
    .appName("Midares test job") \
    .getOrCreate()

24/05/06 15:57:27 WARN Utils: Your hostname, pengfei-Virtual-Machine resolves to a loopback address: 127.0.1.1; using 10.50.2.80 instead (on interface eth0)
24/05/06 15:57:27 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


24/05/06 15:57:28 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
pjc_path= "/home/pengfei/git/PySparkCommonFunc/data/midares/PJC.csv"
pjc_df_1 = spark.read.csv(pjc_path,header=True, inferSchema=True)

In [4]:
pjc_df_1.printSchema()

root
 |-- id_midas: string (nullable = true)
 |-- KROD3: string (nullable = true)
 |-- KDDPJ: timestamp (nullable = true)
 |-- KDFPJ: timestamp (nullable = true)
 |-- KCPJC: integer (nullable = true)
 |-- KCOPP: integer (nullable = true)
 |-- KCFPP: integer (nullable = true)
 |-- KCMNI: string (nullable = true)
 |-- KCALF: double (nullable = true)
 |-- KQRSPP: integer (nullable = true)
 |-- KNOPRP: integer (nullable = true)
 |-- KTDGAP: double (nullable = true)
 |-- KQBPJP: integer (nullable = true)
 |-- KQJSTP: integer (nullable = true)
 |-- KCPJCC: string (nullable = true)
 |-- alloc: string (nullable = true)


In [5]:
print(pjc_df_1.show())

+--------+-----+-------------------+-------------------+-----+-----+-----+-----+-----+------+------+------+------+------+------+-----+
|id_midas|KROD3|              KDDPJ|              KDFPJ|KCPJC|KCOPP|KCFPP|KCMNI|KCALF|KQRSPP|KNOPRP|KTDGAP|KQBPJP|KQJSTP|KCPJCC|alloc|
+--------+-----+-------------------+-------------------+-----+-----+-----+-----+-----+------+------+------+------+------+------+-----+
|     M01|  S01|2017-07-01 00:00:00|2017-09-30 00:00:00|   66|   66|   66|   BK| 26.4|     0|     0|  21.6|     0|    99|    AC| null|
|     M01|  S02|2017-07-04 00:00:00|2017-09-30 00:00:00|   66|   66|   66|   BK| 26.4|     0|     0|  21.6|     0|    99|    AC| null|
|     M01|  S03|2017-07-05 00:00:00|2017-09-30 00:00:00|   66|   66|   66|   BK| 26.4|     0|     0|  21.6|     0|    99|    AC| null|
|     M01|  S04|2017-07-06 00:00:00|2017-09-30 00:00:00|   66|   66|   66|   BK| 26.4|     0|     0|  21.6|     0|    99|    AC| null|
|     M02|  S01|2017-08-05 00:00:00|2017-09-30 00:00:00

In [8]:
pjc_df_1 = pjc_df_1.withColumn("KDDPJ_date",to_date(col("KDDPJ"))).withColumn("KDFPJ_date",to_date(col("KDFPJ"))).drop("KDDPJ","KDFPJ").withColumnRenamed("KDDPJ_date","KDDPJ").withColumnRenamed("KDFPJ_date","KDFPJ")

In [9]:
pjc_df_1.printSchema()

root
 |-- id_midas: string (nullable = true)
 |-- KROD3: string (nullable = true)
 |-- KCPJC: integer (nullable = true)
 |-- KCOPP: integer (nullable = true)
 |-- KCFPP: integer (nullable = true)
 |-- KCMNI: string (nullable = true)
 |-- KCALF: double (nullable = true)
 |-- KQRSPP: integer (nullable = true)
 |-- KNOPRP: integer (nullable = true)
 |-- KTDGAP: double (nullable = true)
 |-- KQBPJP: integer (nullable = true)
 |-- KQJSTP: integer (nullable = true)
 |-- KCPJCC: string (nullable = true)
 |-- alloc: string (nullable = true)
 |-- KDDPJ: date (nullable = true)
 |-- KDFPJ: date (nullable = true)


In [10]:
pjc_df_1.show()

+--------+-----+-----+-----+-----+-----+-----+------+------+------+------+------+------+-----+----------+----------+
|id_midas|KROD3|KCPJC|KCOPP|KCFPP|KCMNI|KCALF|KQRSPP|KNOPRP|KTDGAP|KQBPJP|KQJSTP|KCPJCC|alloc|     KDDPJ|     KDFPJ|
+--------+-----+-----+-----+-----+-----+-----+------+------+------+------+------+------+-----+----------+----------+
|     M01|  S01|   66|   66|   66|   BK| 26.4|     0|     0|  21.6|     0|    99|    AC| null|2017-07-01|2017-09-30|
|     M01|  S02|   66|   66|   66|   BK| 26.4|     0|     0|  21.6|     0|    99|    AC| null|2017-07-04|2017-09-30|
|     M01|  S03|   66|   66|   66|   BK| 26.4|     0|     0|  21.6|     0|    99|    AC| null|2017-07-05|2017-09-30|
|     M01|  S04|   66|   66|   66|   BK| 26.4|     0|     0|  21.6|     0|    99|    AC| null|2017-07-06|2017-09-30|
|     M02|  S01|   66|   66|   66|   BK| 26.4|     0|     0|  21.6|     0|    99|    AC| null|2017-08-05|2017-09-30|
|     M02|  S02|   66|   66|   66|   BK| 26.4|     0|     0|  21

In [11]:
dateformat_str = "%Y-%m-%d"

seuil_debut = datetime.strptime("2017-01-01", dateformat_str)
seuil_fin = datetime.strptime("2023-08-31", dateformat_str)
seuil_raz = datetime.strptime("2021-07-01", dateformat_str)
seuil_fin_censure_17 = datetime.strptime("2018-08-31", dateformat_str)
seuil_fin_censure_18 = datetime.strptime("2019-08-31", dateformat_str)
seuil_fin_censure_19 = datetime.strptime("2020-08-31", dateformat_str)

In [16]:
temp = pjc_df_1.select("id_midas", "KROD3", "KCPJC", "KQBPJP", "KDDPJ", "KDFPJ", "KCFPP", "alloc") \
    .filter(col("KDDPJ") <= seuil_fin) \
    .groupBy("id_midas", "KROD3").agg(max("KDFPJ").alias("date_der_PJC"), min("KDDPJ").alias("date_ouv_droit")).orderBy("id_midas","KROD3")

In [17]:
temp.show()

+--------+-----+------------+--------------+
|id_midas|KROD3|date_der_PJC|date_ouv_droit|
+--------+-----+------------+--------------+
|     M01|  S01|  2017-09-30|    2017-07-01|
|     M01|  S02|  2017-09-30|    2017-07-04|
|     M01|  S03|  2017-09-30|    2017-07-05|
|     M01|  S04|  2017-09-30|    2017-07-06|
|     M02|  S01|  2017-09-30|    2017-08-05|
|     M02|  S02|  2017-09-30|    2017-08-03|
|     M02|  S03|  2017-09-30|    2017-08-02|
|     M02|  S04|  2017-09-30|    2017-08-01|
+--------+-----+------------+--------------+
